In [1]:
using Chemfiles
using DataFrames

In [2]:
function meta_var(s::AbstractString,v::Any)
         s=symbol(s) 
         @eval (($s) = ($v))
end

meta_var (generic function with 1 method)

In [3]:
function tognm(vtor_anm)
    vtor_gnm = Array{Float64}(convert(Int64, length(vtor_anm)/3));
    vtor_anm =  vtor_anm.^2
    for i=1:convert(Int64, length(vtor_anm)/3)
        vtor_gnm[i] = sqrt(vtor_anm[i*3-2] + vtor_anm[i*3-1] + vtor_anm[i*3])
    end
    return vtor_gnm
end

function gpnum(vtor_in::Array{Float64})
    return 1 ./ sum(tognm(vtor_in).^4)
end

function pnum(vtor_in::Array{Float64})
    return 1 ./ sum(vtor_in.^4)
end

pnum (generic function with 1 method)

In [4]:
function read_ptraj_modes(file, modes_elements, norma::Bool=true)    
    modes_file=open(file, "r")
    modes_text = readdlm(modes_file, skipstart=0, skipblanks=true,
    comments=true, comment_char='\*')
    close(modes_file)

    nmodes = modes_text[1, 5]
    ncoords = convert(Int64, modes_elements)
    lines = ceil(Int64, ncoords/7)
    rest = convert(Int64, ncoords % 7)
    
    eval=Array{Float64}(nmodes);
    mode = Array{Float64}(ncoords, nmodes);
    j = lines + 1 + 2 # 1 p/ q lea la prox linea 2 por el header

    for i=1:nmodes
        eval[i] = modes_text[j, 2]
        tmp = map(x -> if x == "" x = 0 else x end, modes_text[(j+1):(lines+j), :])
        mode[:, i] = reshape(transpose(tmp), size(tmp)[1]*size(tmp)[2])[1:modes_elements]
        j = j + lines + 1
    end
    
    if norma == true
        for i=1:nmodes
            mode[: ,i] = mode[:, i] / norm(mode[:, i])
        end
    end
    
    return mode, eval
end

read_ptraj_modes (generic function with 2 methods)

In [12]:
# Get ready
home = "/home/german/labo/17/pdz/"
lb = "run/lb/";
natoms = 2076;
natoms_pdz = 1944;
natoms_lig = natoms - natoms_pdz;

# NEQ trajectory
frame_cnt = 200000;
in_trj_filename = string(home, lb, "pca/", "full_avgfit_lb.nc")
in_trj = Trajectory(in_trj_filename)

# Average.
in_avg_filename = string(home, lb, "pca/", "full_avg_lb.pdb")
in_avg_trj = Trajectory(in_avg_filename)
in_avg_frm = read(in_avg_trj)
in_avg_top = Topology(in_avg_frm)

in_avg_frm_xyz = positions(in_avg_frm);
natoms_system = size(in_avg_frm)
# Obtengo los índices.
ca_indices = [ name(Atom(in_avg_top, i)) for i=0:natoms_system-1 ] .== "CA";
ca_indices_pdz = findn(ca_indices)[1:124]
ca_indices_cut = findn(ca_indices)[12:112]
ca_indices_lig = findn(ca_indices)[125:132]
aa3_cut = length(ca_indices_cut) * 3
aa36_cut = aa3_cut - 6

# Obtengo las posiciones de los CA del average pdz.
ca_avg_xyz = in_avg_frm_xyz[:, ca_indices_cut]
# Obtengo los modos 
t0_modes, t0_evals = read_ptraj_modes(string(home, lb, "pca/full_modes_lb"), aa3_cut);

In [13]:
# Obtengo las posiciones de los CA de la corrida de NEQ y los vectores diferencia respecto
# a su propio average
ca_eq_xyz = Array{Array{Float64, 2}}(frame_cnt)
Vdiff = Array{Array{Float64, 1}}(frame_cnt)
dot_vdiff_modes = Array{Float64, 2}(aa36_cut, frame_cnt)
pnum_t0 = Array{Float64, 1}(frame_cnt)

for frame=1:frame_cnt
    # Obtengo las posiciones de este frame
    in_frm = read_step(in_trj, frame-1)
    in_frms_xyz = positions(in_frm)
    ca_eq_xyz[frame] = in_frms_xyz[:, ca_indices_cut]
    # Obtengo el vector diferencia normalizado de los Calpha no_loop del pdz entre el frame y el avg
    tmp = (reshape(ca_eq_xyz[frame] - ca_avg_xyz, 3 * length(ca_indices_cut)))
    Vdiff[frame] = tmp / norm(tmp)
    # Lo proyecto sobre todos los modos
    for i = 1:aa36_cut
        dot_vdiff_modes[i, frame] = dot(t0_modes[:, i], Vdiff[frame])
    end
end
# Arregla Nans
dot_vdiff_modes[isnan.(dot_vdiff_modes)] = 0

0

In [14]:
# Get Pnumbers
for frame=1:frame_cnt
    pnum_t0[frame] = pnum(dot_vdiff_modes[:, frame])
end

# Round pnums to next natural number
pnum_t0_rounded = convert.(Int64, ceil.(pnum_t0))
# Sort modes according to projection against Vdiff
top_modes = mapslices(x -> sortperm(x, rev =true), abs.(dot_vdiff_modes), 1);
top_modes_indices = map(x -> range(1,x), pnum_t0_rounded);

# Get every mode that appears at least once in the essential subspaces
pnum_indices_t0_each_frame = Array{Int64, 1}(0)
for frame = 1:frame_cnt
    pnum_indices_t0_each_frame = [ pnum_indices_t0_each_frame ; top_modes[:, frame][ top_modes_indices[frame] ]  ]
end
pnum_indices_t0 = sort(unique(pnum_indices_t0_each_frame));

In [15]:
# Now, instead of sorting modes, sort their projections with Vdiff
top_modes_pjtions = mapslices(x -> sort(x, rev =true), abs.(dot_vdiff_modes), 1)

# Get projection values for each mode of each subspacec. Later, they 'll be used as weights
pnum_pjtions_t0_each_frame = zeros(Float64, aa36_cut, frame_cnt)
for frame = 1:frame_cnt
    tmp = top_modes[top_modes_indices[frame], frame]
    for i = 1:length(tmp)
        pnum_pjtions_t0_each_frame[tmp[i], frame] = dot_vdiff_modes[tmp[i], frame]
    end
end

# Con los valores de proyección a c/ frame, puedo sacar el peso de c/ modo p/ todo el paso
weights_t0a = reshape(mapslices(x -> mean(x), pnum_pjtions_t0_each_frame.^2, 2), aa36_cut)

# Ahora obtengo pesos, pero sólamente teniendo en cuenta la frecuencia de aparición en los subespacios
# esenciales de c/ frame
not0_each_frame = copy(pnum_pjtions_t0_each_frame)
not0_each_frame[not0_each_frame .!= 0.] = 1.
weights_t0b = reshape(mapslices(x -> mean(x), not0_each_frame, 2), aa36_cut);

In [17]:
pnum_indices_t0

88-element Array{Int64,1}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
  79
  80
  81
  84
  87
  88
  90
  91
  93
  94
  97
 104

In [10]:
weights_t0a

297-element Array{Float64,1}:
 0.676316   
 0.0827862  
 0.0347803  
 0.0176177  
 0.00843804 
 0.00863517 
 0.0052363  
 0.00455151 
 0.00335161 
 0.00143634 
 0.000930536
 0.000625438
 0.000841085
 ⋮          
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        

## Idem, pero con la corrida sin restraints

In [14]:
# Get ready
home = "/home/german/labo/17/pdz/"
lb = "run/lb/";
natoms = 2076;
natoms_pdz = 1944;
natoms_lig = natoms - natoms_pdz;

# NEQ trajectory
frame_cnt = 80000;
in_trj_filename = string(home, lb, "non_restrained_pca/", "full_avgfit_lb.nc")
in_trj = Trajectory(in_trj_filename)

# Average.
in_avg_filename = string(home, lb, "non_restrained_pca/", "full_avg_lb.pdb")
in_avg_trj = Trajectory(in_avg_filename)
in_avg_frm = read(in_avg_trj)
in_avg_top = Topology(in_avg_frm)

in_avg_frm_xyz = positions(in_avg_frm);
natoms_system = size(in_avg_frm)
# Obtengo los índices.
ca_indices = [ name(Atom(in_avg_top, i)) for i=0:natoms_system-1 ] .== "CA";
ca_indices_pdz = findn(ca_indices)[1:124]
ca_indices_cut = findn(ca_indices)[12:112]
ca_indices_lig = findn(ca_indices)[125:132]
aa3_cut = length(ca_indices_cut) * 3
aa36_cut = aa3_cut - 6

# Obtengo las posiciones de los CA del average pdz.
nonr_ca_avg_xyz = in_avg_frm_xyz[:, ca_indices_cut]
# Obtengo los modos 
nonr_t0_modes, nonr_t0_evals = read_ptraj_modes(string(home, lb, "pca/full_modes_lb"), aa3_cut);

In [16]:
# Obtengo las posiciones de los CA de la corrida de NEQ y los vectores diferencia respecto
# a su propio average
nonr_ca_eq_xyz = Array{Array{Float64, 2}}(frame_cnt)
nonr_Vdiff = Array{Array{Float64, 1}}(frame_cnt)
dot_nonr_Vdiff_modes = Array{Float64, 2}(aa36_cut, frame_cnt)
nonr_pnum_t0 = Array{Float64, 1}(frame_cnt)

for frame=1:frame_cnt
    # Obtengo las posiciones de este frame
    in_frm = read_step(in_trj, frame-1)
    in_frms_xyz = positions(in_frm)
    nonr_ca_eq_xyz[frame] = in_frms_xyz[:, ca_indices_cut]
    # Obtengo el vector diferencia normalizado de los Calpha no_loop del pdz entre el frame y el avg
    tmp = (reshape(nonr_ca_eq_xyz[frame] - ca_avg_xyz, 3 * length(ca_indices_cut)))
    nonr_Vdiff[frame] = tmp / norm(tmp)
    # Lo proyecto sobre todos los modos
    for i = 1:aa36_cut
        dot_nonr_Vdiff_modes[i, frame] = dot(t0_modes[:, i], nonr_Vdiff[frame])
    end
end
# Arregla Nans
dot_nonr_Vdiff_modes[isnan.(dot_nonr_Vdiff_modes)] = 0

0

In [18]:
# Get Pnumbers
for frame=1:frame_cnt
    nonr_pnum_t0[frame] = pnum(dot_nonr_Vdiff_modes[:, frame])
end

# Round pnums to next natural number
nonr_pnum_t0_rounded = convert.(Int64, ceil.(nonr_pnum_t0))
# Sort modes according to projection against nonr_Vdiff
nonr_top_modes = mapslices(x -> sortperm(x, rev =true), abs.(dot_nonr_Vdiff_modes), 1);
nonr_top_modes_indices = map(x -> range(1,x), nonr_pnum_t0_rounded);

# Get every mode that appears at least once in the essential subspaces
nonr_pnum_indices_t0_each_frame = Array{Int64, 1}(0)
for frame = 1:frame_cnt
    nonr_pnum_indices_t0_each_frame = [ nonr_pnum_indices_t0_each_frame ; 
        nonr_top_modes[:, frame][ nonr_top_modes_indices[frame] ]  ]
end
nonr_pnum_indices_t0 = sort(unique(nonr_pnum_indices_t0_each_frame));

In [19]:
# Now, instead of sorting modes, sort their projections with nonr_Vdiff
nonr_top_modes_pjtions = mapslices(x -> sort(x, rev =true), abs.(dot_nonr_Vdiff_modes), 1)

# Get projection values for each mode of each subspacec. Later, they 'll be used as weights
nonr_pnum_pjtions_t0_each_frame = zeros(Float64, aa36_cut, frame_cnt)
for frame = 1:frame_cnt
    tmp = nonr_top_modes[nonr_top_modes_indices[frame], frame]
    for i = 1:length(tmp)
        nonr_pnum_pjtions_t0_each_frame[tmp[i], frame] = dot_nonr_Vdiff_modes[tmp[i], frame]
    end
end

# Con los valores de proyección a c/ frame, puedo sacar el peso de c/ modo p/ todo el paso
nonr_weights_t0a = reshape(mapslices(x -> mean(x), nonr_pnum_pjtions_t0_each_frame.^2, 2), aa36_cut)

# Ahora obtengo pesos, pero sólamente teniendo en cuenta la frecuencia de aparición en los subespacios
# esenciales de c/ frame
nonr_not0_each_frame = copy(pnum_pjtions_t0_each_frame)
nonr_not0_each_frame[not0_each_frame .!= 0.] = 1.
nonr_weights_t0b = reshape(mapslices(x -> mean(x), not0_each_frame, 2), aa36_cut);

### Comparo los subespacios esenciales obtenidos

In [27]:
DataFrame(Restringida = weights_t0a, Libre = nonr_weights_t0a)

,Restringida,Libre
1,0.6761612183630893,0.6761073445822164
2,0.08386651088279434,0.08380720575628398
3,0.03497661069774107,0.03505321675925413
4,0.01806137451370029,0.018051077043892083
5,0.008450985905054704,0.00845228021089399
6,0.008692312509634275,0.008672453613159761
7,0.005250541079648438,0.005252592321065152
8,0.0045590851671689315,0.0045726614066477185
9,0.0033548334562125887,0.00332901630783364
10,0.0014359737581242532,0.0014326535334974856


In [26]:
DataFrame(Restringida = pnum_indices_t0, Libre = nonr_pnum_indices_t0)

,Restringida,Libre
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9
10,10,10
